
# Down, set, hut! Explaining variability in snap timing on plays with motion

---

# Introduction

Consider the following two pass attempts by Kansas City Chiefs quarterback Patrick Mahomes in the 2022 NFL regular season.
The first play (left) is a 9-yard pass to Marquez Valdes-Scantling for a first down against the Colts in week 3.
The second play (right) is a 45-yard TD pass to JuJu Smith-Schuster against the 49ers in week 7.
In both plays, KC (in white) uses the same type of motion—glide, as the motion players (in gold) initially line up in an outside wide alignment before moving laterally toward the inside.

<div style="display: flex;">
<div>
<br>
<center>
<img src="https://raw.githubusercontent.com/qntkhvn/timing/refs/heads/main/figures/kelce_motion.gif">
</center>
</div>
<div>
<br>
<center>
<img src="https://raw.githubusercontent.com/qntkhvn/timing/refs/heads/main/figures/hardman_motion.gif">
</center>
</div>
</div>

In each animation, the solid gold line along the motion player's path (dashed gray line) represents the trajectory portion between the start of motion and the ball-snap.
It is apparent that the plays differ in the duration of this color-coded portion of the path.
In the first play, the snap happens 3.6 seconds after Travis Kelce begins his motion, whereas in the second, less than a second elapses since Mecole Hardman starts motioning.
As both plays result in significant gains (a first down and touchdown), this highlights Mahomes' ability to react, anticipate, and time the snaps in various situations, elevating the effectiveness of the entire KC offensive unit.

Thus, in every play, it is important for the quarterback to synchronize the snap with the team's offensive execution, in order to keep the defense off balance and offense in control.
In this work, **we propose a multilevel modeling framework to assess a passer's ability to adapt and align the snap with their team's pre-snap motion, culminating in a novel approach for measuring variability in QB snap timing.**

# Defining snap timing

We define the snap timing $\delta$ for a play as the number of frames elapsed between the start of motion $(t_{\text{motion}})$ and the ball-snap $(t_{\text{snap}}).$
That is,$$\delta=t_{\text{snap}}-t_{\text{motion}}\,.$$ 
Note that in our analysis, we only consider **plays with receivers in motion at snap and also running a route.**

Here, it is straightforward to obtain $t_{\text{snap}}$ for each play using the frame label in the tracking data.
Conversely, identifying $t_{\text{motion}}$ requires more effort, due to [inconsistency between player-level motion indicators and annotated tracking events](https://www.kaggle.com/competitions/nfl-big-data-bowl-2025/discussion/543709).

We use the following procedure to determine $t_{\text{motion}}$ for each play.

-   For plays where the player in motion at snap is also the only player in motion since line-set, we use the `man_in_motion` event frame in the tracking data as the start of motion.

-   For the remaining plays (specifically, plays where motion player at snap is not the sole player in motion since line-set, and plays with players charted as in motion at snap but without a `man_in_motion` tracking event):

    -   We first observe the distribution for the ratio between the speed of motion player at `man_in_motion` and their maximum speed between line-set and snap (only using plays with $t_{\text{motion}}$ identified earlier). 

    -   We then choose the average ratio value of 0.45 as the threshold for determining the start of motion for the remaining plays. Specifically, we assign $t_{\text{motion}}$ as the first frame between line-set and snap where the player reaches at least 45% of their top speed.

# Multilevel model for snap timing

## Model specification

The observed timing between motion and snap on a single play is surely attributable to numerous variables—from contextual and spatiotemporal features, to the players and teams involved.
To this end, we fit the following multilevel model:
$$
\begin{aligned}
\delta_i&\sim\textsf{Gamma}(\mu_i,\alpha_i)\\
\\
\log\mu_i&=\gamma_0+\boldsymbol{\beta X_i}+b_{q[i]}+b_{m_[i]}+b_{d[i]}\\
b_q&\sim\textsf{Normal}(0,\sigma^2_q)\\
b_m&\sim\textsf{Normal}(0,\sigma^2_m)\\
b_d&\sim\textsf{Normal}(0,\sigma^2_d)\\
\\
\log\alpha_i&=\psi_0+u_{q[i]}\\
u_q&\sim\textsf{Normal}(0,\tau^2_q)\\
\end{aligned}
$$
In detail, we model the response $\delta_i>0$ (snap timing for play $i$) using a Gamma distribution (with log link function,
[parameterized by mean $\mu$ and shape $\alpha$](https://paulbuerkner.com/brms/articles/brms_families.html#:~:text=The%20density%20of%20the%20Gamma,shape%20parameter)),
which is well-suited for modeling a positive, continuous ("interarrival time") variable like our case.

Of primary interest, we model the shape $\alpha$ (which is proportional to the Gamma distribution's variance) with QB random effects.
This allows us to examine the variability in timing between motion and snap among NFL quarterbacks.
From an offensive standpoint, we hypothesize that a higher timing variability is beneficial because it prevents defenses from predicting when the snap will occur.
This may allow the offense to control the game flow and exploit defensive vulnerabilities.

We also model the mean parameter $\mu$ by including random intercepts for three groups: quarterback $q$, motion player $m$, and defensive team $d$.
Moreover, we account for contextual and personnel information about play $i$ through $\boldsymbol{X_i}$, and estimate the coefficients $\boldsymbol\beta$ as fixed effects.
These include the current down, number of timeouts remaining for the offense, play clock at the start of motion, motion player's position, number of players in motion since line-set, and type of motion, as they all can impact the timing between motion and snap.

It is noteworthy that the motion type is a vital feature for modeling snap timing, since it relates to play design.
For instance, in a play-action pass, a team can fake a jet sweep to set up the QB for a pass.
However, this information is not provided in the data, prompting us to consider a clustering algorithm to identify the different types of motion.

## Motion type clustering

Our clustering input contains tracking data features describing the motion player's location at different events relative to where the center lines up before the snap.
The following figure illustrates the derived features for the example play from earlier with Travis Kelce in motion.
We compute the displacement between the center and motion player at ball-snap in both sideline $(\textsf{B}_{\textsf{sideline}})$ and endzone $(\textsf{B}_{\textsf{endzone}})$ directions.
We also include the displacement with respect to the sideline between the center and motion player at the start of motion $(\textsf{M}_{\textsf{sideline}})$ as well as when they cross the line of scrimmage (LOS) $(\textsf{L}_{\textsf{sideline}})$.

<center>
<img src="https://raw.githubusercontent.com/qntkhvn/timing/refs/heads/main/figures/clustering_features.png" style="width:60%">
</center>

Note that for plays where the motion player never crosses the LOS, we instead use their location at either the QB event (e.g., forward pass, sack) or 3 seconds after the snap (depending on which happens sooner) in place of $\textsf{L}_{\textsf{sideline}}$.
The aforementioned 3-second threshold is chosen using the observed distribution of the time elapsed from snap to crossing LOS for plays where motion players do cross the LOS, as it captures a sufficiently large fraction (~95%) of the values.

We fit a Gaussian mixture model via the `mclust` `R` package to obtain unsupervised labels for the pre-snap motions.
We select the optimal number of Gaussian clusters to be $G=6$ using BIC, and validate the cluster assignment through visual inspection of player motion trajectories in each cluster.
Ultimately, we condition on these cluster labels as fixed effects in modeling the mean snap timing $\mu$, so that our estimates of QB snap timing variability $u_q$ (at the shape-level) are not confounded by different types of motion.

# Analysis

Our multilevel model is fit in a Bayesian framework via the `brms` `R` package using four parallel chains of 10,000 iterations and 5,000 burn-in draws.
We perform diagnostics by inspecting trace plots as well as computing the $\widehat{R}$ and effective sample size statistics, and find no issues with model convergence and efficiency.
Our Bayesian approach provides natural uncertainty quantification for our model parameters via their posterior distributions.
As it turns out, among the considered random effects, the largest source variation is between QBs when modeling the snap timing shape (captured by $\tau_q^2$).
Thus, we focus on the QB shape random effects $u_q$ in the following analysis.

## Leaderboard

The following figure displays the posterior distributions for the shape random effect $u_q$ (ordered by posterior mean) for QBs with at least 50 pass attempts across the considered motion plays.
For each player, the posterior mean value provides an estimate for their ability to maintain variability in snap timing, **with higher posterior means corresponding to greater variability in timing between the start of motion and snap.**

Patrick Mahomes stands out as the top QB according our measure, adding the ability to time snaps as another element to his success.
Other high-caliber QBs such as Tom Brady and Josh Allen are also among the leaders in our rankings.
Note that there is considerable uncertainty in our estimates, as demonstrated by the rather wide credible intervals (CIs), which is unsurprising given our limited sample of motion plays.
Still, we observe substantial differences in the CIs across this subset of QBs.
For instance, the 95% CI for Daniel Jones (who ranks last in our leaderboard) does not overlap with the top five quarterbacks' 95% CIs.

<center>
<img src="https://raw.githubusercontent.com/qntkhvn/timing/refs/heads/main/figures/posteriors_slab.png" style="width:65%">
</center>

## Comparison to havoc rate

We now examine the relationship between the posterior mean for the QB shape random effect $u_q$ and a measure of "havoc rate".
Here, a havoc event is defined as whether any of the following defensive outcomes are generated on a play: pass breakup, forced fumble, tackle for loss, interception, sack, and pressure.
The figure below is a scatterplot of the posterior mean for the shape random effect and havoc rate across all passing plays over the first nine weeks of the 2022 NFL season (left) and only the considered motion plays (right) for the same subset of QBs as before.

**We observe that lower snap timing variability corresponds to higher rate of facing havoc events created by the opposing defense.**
This makes intuitive sense, as when there is little variability in timing, the offense is likely to be predictable and experience unfavorable play outcomes.
In contrast, by varying the duration between motion and snap, offenses can create uncertainty, forcing defenders to play more cautiously which reduces their effectiveness in executing disruptive plays.

<center>
<img src="https://raw.githubusercontent.com/qntkhvn/timing/refs/heads/main/figures/corr_havoc.png" style="width:99%">
</center>

As a side note, the posterior mean for the shape random effect scarcely correlates with the rate of motion on all passing plays $(r = 0.11)$ over the first nine weeks of the 2022 season.
Additionally, there is a weak correlation between motion rate and havoc rate $(r = -0.09)$.
This suggests that **our measure of QB snap timing variability provides independent information of motion tendency, while also being a more direct indicator of encountering defensive disruptions than the prevalence of motion.**

# Discussion

Variability in snap timing is an essential aspect for an offense to dictate plays and make it difficult for defenses to anticipate and react.
Through multilevel modeling, we provide an assessment of a quarterback's ability to maintain variable snap timing with their teammates' motion.
We demonstrate that higher variability in snap timing is beneficial for the passing game, as it relates to experiencing less disruptions generated by the defense.
Although we solely focus on snap timing for passing plays with motion, our estimates may also serve as proxies for QB awareness or pocket presence, as indicated by the moderate correlation with havoc rate across all passing plays (not just the plays considered in our model).

Our proposed framework is not without limitations.
First, to identify the start of motion, we use a threshold-based criterion which relies on the moment at which the motion player achieves a certain fraction of their top speed prior to the snap.
This is certainly a simple viewpoint, as one could come up with a more robust definition by building a model to predict the starting motion frame.
Further, despite the reasonable set of inputs in our cluster analysis, there is possibly room for improvement in identifying the motion types by having more refined features, or alternatively using a functional clustering approach.
Since neither of these tasks is the main focus of our work, we leave these explorations for the future.

Additionally, our study is limited to only passing plays with receivers in motion at snap.
This introduces a selection bias into our analysis by excluding running plays and various QBs who do not meet the cutoff for minimum pass attempts on motion plays (e.g., Jalen Hurts and Joe Burrow).
We also recognize that while our estimates are referred to as "QB random effects", they are certainly correlated with the team's center and coach's play-calling.
Nevertheless, **we believe our contribution provides a novel description of snap timing variability and reveals insight into an understudied area of football.**


# Appendix

All code is available at https://github.com/qntkhvn/timing.

RY thanks Sam Schwartzstein for a conversation which motivated the topic of QB snap timing variability.
QN thanks Nick Wan whose [Twitch stream](https://www.twitch.tv/nickwan_datasci) and [Kaggle notebook](https://www.kaggle.com/code/nickwan/fly-sweeps-are-the-best-rushing-play) inspired the definition and identification of motion in this work.

On a personal note, QN failed to convince RY to name the metric *STRAIN (Patrick's version)*, where STRAIN stands for Snap Timing Rhythm and Adaptability INdex.

---

[Quang Nguyen](https://github.com/qntkhvn) and [Ron Yurko](https://github.com/ryurko)

Department of Statistics & Data Science

Carnegie Mellon University